# Data Merging
Notebook to merge data from provided files

In [ ]:
# Import libraries necessary for this project
import numpy as np
import pandas as pd

# Allows the use of display() for DataFrames
from IPython.display import display

import seaborn as sns
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

import missingno as msno

from utils import *

In [ ]:
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import gc
gc.collect()

## Ext Bureau

In [ ]:
bureau = pd.read_csv('input/bureau.csv.zip')
bureau.head(2)

In [ ]:
bureau_balance = pd.read_csv('input/bureau_balance.csv.zip')
display(bureau_balance.head(2))

Merging bureau and balance data

In [ ]:
bureau_balance_by_id = bureau_balance.groupby('SK_ID_BUREAU')
bureau_grouped_size = bureau_balance_by_id['MONTHS_BALANCE'].size()
bureau_grouped_max = bureau_balance_by_id['MONTHS_BALANCE'].max()
bureau_grouped_min = bureau_balance_by_id['MONTHS_BALANCE'].min()

bureau_counts = bureau_balance_by_id['STATUS'].value_counts(normalize = False)
bureau_counts_unstacked = bureau_counts.unstack('STATUS')
bureau_counts_unstacked.columns = ['STATUS_0', 'STATUS_1','STATUS_2','STATUS_3','STATUS_4','STATUS_5','STATUS_C','STATUS_X',]
bureau_counts_unstacked['MONTHS_COUNT'] = bureau_grouped_size
bureau_counts_unstacked['MONTHS_MIN'] = bureau_grouped_min
bureau_counts_unstacked['MONTHS_MAX'] = bureau_grouped_max

bureau = bureau.join(bureau_counts_unstacked, how='left', on='SK_ID_BUREAU')

bureau = pd.get_dummies(bureau)

In [ ]:
bureau_by_skid = bureau.groupby('SK_ID_CURR')
avg_bureau = bureau_by_skid.mean()
avg_bureau['buro_count'] = bureau[['SK_ID_BUREAU', 'SK_ID_CURR']].groupby('SK_ID_CURR').count()['SK_ID_BUREAU']

In [ ]:
avg_bureau.shape

In [ ]:
installments_payments = pd.read_csv('input/installments_payments.csv.zip')
installments_payments.head(2)

In [ ]:
credit_card_balance = pd.read_csv('input/credit_card_balance.csv.zip')
credit_card_balance.head(2)

In [ ]:
POS_CASH_balance = pd.read_csv('input/POS_CASH_balance.csv.zip')
POS_CASH_balance.head(2)

In [ ]:
previous_application = pd.read_csv('input/previous_application.csv.zip')
previous_application.head(2)

In [ ]:
app_train = pd.read_csv('input/application_train.csv.zip')
app_test = pd.read_csv('input/application_test.csv.zip')

In [ ]:
app_train = data.merge(right=avg_prev.reset_index(), how='left', on='SK_ID_CURR')
app_test = test.merge(right=avg_prev.reset_index(), how='left', on='SK_ID_CURR')

data = data.merge(right=avg_buro.reset_index(), how='left', on='SK_ID_CURR')
test = test.merge(right=avg_buro.reset_index(), how='left', on='SK_ID_CURR')

data = data.merge(POS_CASH.groupby('SK_ID_CURR').mean().reset_index(), how='left', on='SK_ID_CURR')
test = test.merge(POS_CASH.groupby('SK_ID_CURR').mean().reset_index(), how='left', on='SK_ID_CURR')

data = data.merge(credit_card.groupby('SK_ID_CURR').mean().reset_index(), how='left', on='SK_ID_CURR')
test = test.merge(credit_card.groupby('SK_ID_CURR').mean().reset_index(), how='left', on='SK_ID_CURR')

data = data.merge(right=avg_payments.reset_index(), how='left', on='SK_ID_CURR')
test = test.merge(right=avg_payments.reset_index(), how='left', on='SK_ID_CURR')

data = data.merge(right=avg_payments2.reset_index(), how='left', on='SK_ID_CURR')
test = test.merge(right=avg_payments2.reset_index(), how='left', on='SK_ID_CURR')

data = data.merge(right=avg_payments3.reset_index(), how='left', on='SK_ID_CURR')
test = test.merge(right=avg_payments3.reset_index(), how='left', on='SK_ID_CURR')

del avg_prev, avg_buro, POS_CASH, credit_card, avg_payments, avg_payments2, avg_payments3
gc.collect()

print ('Training Data ', data.shape)
print ('Test Data', test.shape)
print ('Target Data', y.shape)